In [51]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SequentialFeatureSelector
import pretty_midi
import warnings
import os
import import_midi as im

In [52]:
def get_genres(path):
    """
    This function reads the genre labels and puts it into a pandas DataFrame.
    
    @input path: The path to the genre label file.
    @type path: String
    
    @return: A pandas dataframe containing the genres and midi IDs.
    @rtype: pandas.DataFrame
    """
    ids = []
    genres = []
    with open(path) as f:
        line = f.readline()
        while line:
            if line[0] != '#':
                [x, y, *_] = line.strip().split("\t")
                ids.append(x)
                genres.append(y)
            line = f.readline()
    genre_df = pd.DataFrame(data={"Genre": genres, "TrackID": ids})
    return genre_df

# Get the Genre DataFrame
genre_path = "new_labels_full.cls"
#genre_path = "new_labels_small.cls"
genre_df = get_genres(genre_path)

# Create Genre List and Dictionary
label_list = list(set(genre_df.Genre))
label_dict = {lbl: label_list.index(lbl) for lbl in label_list}

# Print to Visualize
print(genre_df.head(), end="\n\n")
print(label_list, end="\n\n")
print(label_dict, end="\n\n")

  Genre                               TrackID
0   Pop      5ive_-_Dont_Wanna_Let_You_Go.mid
1   Pop          5ive_-_If_Ya_Gettin_Down.mid
2   Pop             5ive_-_Keep_On_Moving.mid
3   Pop          5ive_-_Slam_Dunk_Da_Funk.mid
4   Pop  5ive_-_Until_The_Time_Is_Through.mid

['Pop', 'Jazz', 'Classic']

{'Pop': 0, 'Jazz': 1, 'Classic': 2}



In [53]:
#This function is used when all the tracks are in one folder
def get_matched_midi_single(midi_folder, genre_df):
    """
    This function loads in midi file paths that are found in the given folder, puts this data into a
    pandas DataFrame, then matches each entry with a genre described in get_genres.
    
    @input midi_folder: The path to the midi files.
    @type midi_folder: String
    @input genre_df: The genre label dataframe generated by get_genres.
    @type genre_df: pandas.DataFrame
    
    @return: A dataframe of track id and path to a midi file with that track id.
    @rtype: pandas.DataFrame
    """
    track_ids, file_paths = [], []
    for filename in os.listdir(midi_path):

        #IMPORTANT:
        track_id = filename
        #YOU'LL NEED TO CHANGE THIS WHEN USING LARGE DATASET

        file_path = midi_path + "/" + filename
        track_ids.append(track_id)
        file_paths.append(file_path)
    all_midi_df = pd.DataFrame({"TrackID": track_ids, "Path": file_paths})
    df = pd.merge(all_midi_df, genre_df, on='TrackID', how='inner')
    return df.drop(["TrackID"], axis=1)

# Obtain DataFrame with Matched Genres to File Paths
midi_path = "data_onefile"
#midi_path = "replacement_data"
matched_midi_df_replace = get_matched_midi_single(midi_path, genre_df)

# Print to Check Correctness
print(matched_midi_df_replace.head())

                                   Path    Genre
0  classifier/data_onefile/003706b_.mid  Classic
1  classifier/data_onefile/003806b_.mid  Classic
2  classifier/data_onefile/003907bv.mid  Classic
3  classifier/data_onefile/003907b_.mid  Classic
4  classifier/data_onefile/004003b_.mid  Classic


In [54]:
%%time
def normalize_features(features):
    """
    This function normalizes the features to the range [-1, 1]
    
    @input features: The array of features.
    @type features: List of float
    
    @return: Normalized features.
    @rtype: List of float
    """
    tempo = (features[0] - 150) / 300
    num_sig_changes = (features[1] - 2) / 10
    resolution = (features[2] - 260) / 400
    time_sig_1 = (features[3] - 3) / 8
    time_sig_2 = (features[4] - 3) / 8
    return [tempo, resolution, time_sig_1, time_sig_2]


def get_features(path):
    """
    This function extracts the features from a midi file when given its path.
    
    @input path: The path to the midi file.
    @type path: String
    
    @return: The extracted features.
    @rtype: List of float
    """
    try:
        file = pretty_midi.PrettyMIDI(path)
        
        tempo = file.estimate_tempo()
        num_sig_changes = len(file.time_signature_changes)
        resolution = file.resolution
        ts_changes = file.time_signature_changes
        ts_1 = 4
        ts_2 = 4
        if len(ts_changes) > 0:
            ts_1 = ts_changes[0].numerator
            ts_2 = ts_changes[0].denominator
        return normalize_features([tempo, num_sig_changes, resolution, ts_1, ts_2])
    except:
        return None

def extract_midi_features(path_df):
    """
    This function takes in the path DataFrame, then for each midi file, it extracts certain
    features, maps the genre to a number and concatenates these to a large design matrix to return.
    
    @input path_df: A dataframe with paths to midi files, as well as their corresponding matched genre.
    @type path_df: pandas.DataFrame
    
    @return: A matrix of features along with label.
    @rtype: numpy.ndarray of float
    """
    all_features = []
    for index, row in path_df.iterrows():
        features = get_features(row.Path)
        genre = label_dict[row.Genre]
        if features is not None:
            features.append(genre)
            all_features.append(features)
    return np.array(all_features)

labeled_features = extract_midi_features(matched_midi_df_replace)
print(labeled_features)

C:\Users\leois\AppData\Local\Programs\Python\Python39\lib\site-packages\pretty_midi\pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
[[ 0.11333419  1.91        0.125       0.125       2.        ]
 [-0.13428571  1.91        0.125       0.125       2.        ]
 [-0.03066667  1.91        0.125       0.125       2.        ]
 ...
 [ 0.05458534 -0.41        0.125       0.125       0.        ]
 [ 0.17007582 -0.35        0.125       0.125       0.        ]
 [ 0.15167655 -0.35        0.125       0.125       0.        ]]
Wall time: 12min 51s


In [55]:
# Shuffle Entire Dataset to Make Random
labeled_features = np.random.permutation(labeled_features)

# Partition into 3 Sets
num = len(labeled_features)
num_training = int(num * 0.8)
num_validation = int(num * 0.9)
training_data = labeled_features[:num_training]
validation_data = labeled_features[num_training:num_validation]
test_data = labeled_features[num_validation:]

# Separate Features from Labels
num_cols = training_data.shape[1] - 1
training_features = training_data[:, :num_cols]
validation_features = validation_data[:, :num_cols]
test_features = test_data[:, :num_cols]

# Format Features for Multi-class Classification
num_classes = len(label_list)
training_labels = training_data[:, num_cols].astype(int)
validation_labels = validation_data[:, num_cols].astype(int)
test_labels = test_data[:, num_cols].astype(int)

# Function for One-Hot Encoding
def one_hot(labels):
    """
    This function encodes the labels using one-hot encoding.
    
    @input num_classes: The number of genres/classes.
    @type num_classes: int
    @input labels: The genre labels to encode.
    @type labels: numpy.ndarray of int
    
    @return: The one-hot encoding of the labels.
    @rtype: numpy.ndarray of int
    """
    return np.eye(num_classes)[labels].astype(int)

# Print to Check Dimentions and to Visualize
print(test_features[:10])
print(test_labels[:10])
print(one_hot(test_labels)[:10])

[[ 0.07333368 -0.41        0.125       0.125     ]
 [ 0.02840157 -0.53        0.125       0.125     ]
 [ 0.22155298 -0.17        0.          0.125     ]
 [-0.08095273 -0.35        0.125       0.125     ]
 [ 0.21095911  0.31        0.125       0.125     ]
 [ 0.15113349 -0.35        0.125       0.125     ]
 [ 0.23704356  0.55        0.125       0.125     ]
 [ 0.12666654 -0.35        0.125       0.125     ]
 [ 0.26896419 -0.17        0.125       0.125     ]
 [ 0.18022747 -0.35        0.125       0.125     ]]
[1 0 2 2 0 1 0 0 2 1]
[[0 1 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]]


In [65]:
def train_model(t_features, t_labels, v_features, v_labels):
    """
    This function trains a neural network using a couple different configurations.
    
    @input t_features: The training features.
    @type t_features: numpy.ndarray of float
    @input t_labels: The training labels.
    @type t_labels: numpy.ndarray of int
    @input v_features: The validation features.
    @type v_features: numpy.ndarray of float
    @input v_labels: The validation labels.
    @type v_labels: numpy.ndarray of int
    
    @return: The classifier that achieved the best validation accuracy.
    @rtype: sklearn.neural_network.multilayer_perceptron.MLPClassifier
    """
    # Neural Network and SVM Configurations
    clf_1 = MLPClassifier(solver='adam', alpha=1e-4, hidden_layer_sizes=(5,), random_state=1)
    clf_2 = MLPClassifier(solver='adam', alpha=1e-4, hidden_layer_sizes=(5, 5), random_state=1)
    clf_3 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=1)
    clf_4 = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 100), random_state=1)
    clf_svm = SVC()
    
    # Keep Track of the Best Model
    best_clf = None
    best_accuracy = 0
    
    # Test the Accuracies of the Models and Get Best
    for clf in [clf_1, clf_2, clf_3, clf_4, clf_svm]:
        t_labels_hot = one_hot(t_labels)
        v_labels_hot = one_hot(v_labels)
        if (type(clf) == SVC):
            clf = clf.fit(t_features, t_labels)
        else:
            clf = clf.fit(t_features, t_labels_hot)
        predictions = clf.predict(v_features)
        count = 0
        for i in range(len(v_labels)):
            if (type(clf) != SVC):
                if np.array_equal(v_labels_hot[i], predictions[i]):
                    count += 1
            else:
                if v_labels[i] == predictions[i]:
                    count += 1
        accuracy = count / len(v_labels_hot)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_clf = clf

    print("Best Accuracy:", best_accuracy)
    return best_clf

classifier = train_model(training_features, training_labels, validation_features, validation_labels)

C:\Users\leois\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\leois\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\leois\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\leois\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimiza

In [76]:
def calculate_accuracy(clf, t_features, t_labels):
    """
    This function takes a trained model as well as the test features and its
    corresponding labels, and reports the accuracy of the model.
    
    @input clf: The trained classifier.
    @type model: sklearn.neural_network.multilayer_perceptron.MLPClassifier
    @input t_features: The features from the test set.
    @type f_features: numpy.ndarray of float
    @input t_labels: The labels of the test set features.
    @type t_labels: numpy.ndarray of int
    
    @return: The accuracy.
    @rtype: float
    """
    count = 0
    predictions = clf.predict(t_features)
    t_labels_hot = one_hot(t_labels)
    for i in range(len(t_features)):
        if (type(clf) == SVC):
            if t_labels[i] == predictions[i]:
                count += 1
        else:
            if np.array_equal(t_labels_hot[i], predictions[i]):
                count += 1
    return count / len(t_features)

# Print the Test Accuracy
print(calculate_accuracy(classifier, test_features, test_labels))

0.6966666666666667


In [72]:
def make_prediction(clf, midi_path):
    """
    This function uses the classifier to predict the genre of a midi file.
    
    @input clf: The trained classifier.
    @type clf: sklearn.neural_network.multilayer_perceptron.MLPClassifier
    @input midi_path: The path to the midi file that we are trying to classify.
    @type midi_path: String
    
    @return: The predicted genre of the midi file.
    @rtype: String
    """
    features = get_features(midi_path)
    #if the classifier doesn't know it
    prediction_ind = clf.predict([features])
   
    prediction = label_list[prediction_ind[0]]
    return prediction
    
# Make a Prediction
test_midi_path ="replacement_data/c1.mid"
print(make_prediction(classifier, test_midi_path))

Classic


In [73]:
def classify_folder(clf, midi_path):
    """
    This function uses the classifier to predict the genre of a midi folder.
    
    @input clf: The trained classifier.
    @type clf: sklearn.neural_network.multilayer_perceptron.MLPClassifier
    @input midi_path: The path to the midi folder that we are trying to classify.
    @type midi_path: String
    
    @return: The predicted genre of the midi file.
    @rtype: String
    """
    #for printing to file
    #sys.stdout = open("classifier_output.txt", "w")
    for filename in os.listdir(midi_path):
        file_path_list = midi_path + "/" + filename
        output = make_prediction(clf, file_path_list)
        print(filename+ ":" +output)
    #sys.stdout.close()

In [75]:
classify_folder(classifier, "replacement_data")

c1.mid:Classic
c2.mid:Pop
c3.mid:Classic
c4.mid:Classic
c5.mid:Classic
c6.mid:Classic
c7.mid:Classic
c8.mid:Classic
c9.mid:Classic
j1.mid:Pop
j2.mid:Pop
j3.mid:Pop
j4.mid:Pop
j5.mid:Pop
j6.mid:Pop
j7.mid:Pop
j8.mid:Pop
j9.mid:Pop
p1.mid:Pop
p2.mid:Pop
p3.mid:Pop
p4.mid:Pop
p5.mid:Pop
p6.mid:Pop
p7.mid:Pop
p8.mid:Pop
p9.mid:Pop


In [5]:
def make_labels(genre, path):
    #for printing to file
    sys.stdout = open("classifier/new_labels_small.cls", "w",encoding='utf-8')
    for filename in os.listdir(path):
        print(filename +'\t'+genre)
    sys.stdout.close()